In [1]:
import wave
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import math
import librosa
import json
import wave
import sys
import pickle
import sklearn

import urllib.request

import librosa.display
import scipy, matplotlib.pyplot as plt, IPython.display as ipd

from scipy.io import wavfile

import librosa.display
import soundfile as sf

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import os
import matplotlib.style as ms
from tqdm import tqdm
import random
ms.use('seaborn-muted')
%matplotlib inline

## Audio dataset

In [21]:
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, sampling_rate=44100, wav_files_path='Audio_preprocess/', meta_data="df_prep.csv", shuffle=False):
        self.emotion_dict = {'ang': 0,
                             'dis': 1,
                             'hap': 2,
                             'sad': 3,
                             'sca': 4,
                             'sur': 5,
                             'neu': 6
                            }
        
        self.wav_files_path = wav_files_path
        self.sr = sampling_rate
        self.shuffle = shuffle
        self.meta_data = pd.read_csv(meta_data).to_numpy()
        
    def getAudio(self, file_path):
        wav_vector, _sr = librosa.load(file_path, sr=self.sr)
        """
        TODO: implement later needed processing of wav vector
        """
        wav_vector = torch.from_numpy(wav_vector)
        return wav_vector
                
    def __getitem__(self, index):
        file_path = self.meta_data[index][0]
        emotion = self.emotion_dict[self.meta_data[index][1]]
        emotion = torch.tensor(emotion)
        return self.getAudio(file_path), emotion

    def __len__(self):
        return len(self.meta_data)

In [22]:
dataset = AudioDataset()
len(dataset)

10848

In [23]:
# dataset = AudioDataset()
train_loader = DataLoader(dataset, num_workers=1, shuffle=False,
                              batch_size=1)

for batch in train_loader:
    print("!!")

!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!!
!

KeyboardInterrupt: 

In [3]:
def train_cycle(model, n_epoch, train_loader, validation_loader, device):
    model.train()
    epoch_losses = []
    epoch_losses_val = []
    for epoch in range(n_epoch):
        print("Epoch: {}".format(epoch))
        losses = []
        for i, batch in enumerate(train_loader):
            model.zero_grad()
            optimizer.zero_grad()
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            
            y_pred = model(x)
            y_pred = y_pred.to(device)
            
            loss = loss_func(y_pred, y)
            loss.backward()
            optimizer.step()
            
            losses.append(loss.item())
        epoch_losses.append(np.mean(losses))
        
#         fig = plt.figure(figsize=(14,5))
#         ax1 = fig.add_subplot(121)
#         plt.plot(epoch_losses)
#         plt.title("Loss")
        
        validation_losses = []
        with torch.no_grad():
            for i, batch in enumerate(validation_loader):
                x, y = batch
                x = x.to(device)
                y = y.to(device)

                y_pred = model(x)
                y_pred = y_pred.to(device)

                loss = loss_func(y_pred, y)
                validation_losses.append(loss.item())
            epoch_losses_val.append(np.mean(losses))
    return epoch_losses, epoch_losses_val, model

In [4]:
def train(model, dataset, loss_func=None, lr=0.001, n_epoch=1000, batch_size=4, shuffle=True):
    if loss_func is None:
        loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    
    print("Loading data")
    validation_split = .15
    shuffle_dataset = shuffle
    random_seed= 42

    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset :
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]
    
    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    
    
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=1,
                                               sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=1,
                                                    sampler=valid_sampler)
    try:
        train_loss, val_loss, best_model = train_cycle(model, n_epoch, train_loader, validation_loader, device)
    except KeyboardInterrupt:
        print("Keyboard interrupt, continue work")
        return
    except:
        print("Something went wrong")
        raise
    return train_loss, val_loss, best_model

In [5]:
class LSTMClassifier(nn.Module):
    """docstring for LSTMClassifier"""
    def __init__(self, config):
        super(LSTMClassifier, self).__init__()
        self.n_layers = config['n_layers']
        self.input_dim = config['input_dim']
        self.hidden_dim = config['hidden_dim']
        self.output_dim = config['output_dim']
        self.bidirectional = config['bidirectional']
        self.dropout = config['dropout'] if self.n_layers > 1 else 0

        self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, bias=True,
                           num_layers=2, dropout=self.dropout,
                           bidirectional=self.bidirectional)
        self.out = nn.Linear(self.hidden_dim, self.output_dim)
        self.softmax = F.softmax

    def forward(self, input_seq):
        # input_seq =. [1, batch_size, input_size]
        rnn_output, (hidden, _) = self.rnn(input_seq)
        if self.bidirectional:  # sum outputs from the two directions
            rnn_output = rnn_output[:, :, :self.hidden_dim] +\
                        rnn_output[:, :, self.hidden_dim:]
        class_scores = F.softmax(self.out(rnn_output[0]), dim=1)
        return class_scores

In [6]:
config = {
    'gpu': 0,
    'bidirectional': True,
    'input_dim': 6,
    'hidden_dim': 50,
    'output_dim': 7,  # number of classes
    'dropout': 0.2,
    'learning_rate': 0.001,
    'batch_size': 512,  # carefully chosen
    'n_epochs': 55000,
    'n_layers': 2,
    'model_code': 'basic_lstm'
}

In [ ]:
model = LSTMClassifier(config)
dataset = AudioDataset()

In [ ]:
train_loader = DataLoader(dataset, num_workers=1, shuffle=True,
                              batch_size=4)

for batch in train_loader:
    print("!!")

In [8]:
train(model, dataset)

Loading data
Epoch: 0
Something went wrong


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/olegs/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/olegs/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/olegs/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 79, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/olegs/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 79, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/olegs/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 64, in default_collate
    return default_collate([torch.as_tensor(b) for b in batch])
  File "/home/olegs/anaconda3/lib/pyth